# Tiny flowing Tensors
Implementing a small version of TensorFlow with the help of Udacity, to better understand backpropagation and differentiable graphs.

In [44]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        #print("ibn: {}".format(inbound_nodes))
        # Inbound Nodes are those that the current node receives values from
        # I'm passing a LIST of NODE OBJECTS
        self.inbound_nodes = inbound_nodes
        # Outbound Nodes are those that it passes the values forward to
        self.outbound_nodes = []
        # For every inbound node add an outbound node to that node
        # TODO: I don't quite get it. Practice coming up...!
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        # the value the node will eventually calculate
        # oh wow. My problem was that I kept this initial setting to 0 of the self.value...
        #self.value = None
        
    
    def forward(self):
        """
        Forward Propagation.
        
        Compute the output based on `inbound_nodes`
        and store the result in self.value
        """
        # implemented in the subclasses
        raise NotImplemented

In [30]:
class Input(Node):
    def __init__(self):
        # no inbound nodes for an input node
        # therefore the instantiator doesn't need inputs
        Node.__init__(self)
        
    def forward(self, value=None):
        # input nodes get a value input by Mr. Human
        # either explicitly or through this forward() method:
        if value is not None:
            self.value = value

In [48]:
class Add(Node):
    def __init__(self, node1, node2):
        Node.__init__(self, [node1, node2])
    
    # remember to pass `self` into the class methods!!
    def forward(self):
        """Calculates the sum of the values of the inbound nodes."""
        value_node1 = self.inbound_nodes[0].value
        value_node2 = self.inbound_nodes[1].value
        self.value = value_node1 + value_node2

In [32]:
# fancy functions defined by Udacity, that I am a benefactor of

def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node
    and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value


In [47]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y
"""

x, y = Input(), Input()

f = Add(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))


10 + 5 = 20 (according to miniflow)
